![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
# Programming Quantum Computers by O'Reilly Media -  [book info](http://shop.oreilly.com/product/0636920167433.do)  - [all code samples](https://oreilly-qc.github.io)

## Code samples for Chapter 9
These code samples were written by Mariia Mykhailova.

### Example 9-1: Using a QPU to increment a number in RAM

In [1]:
// Example 9-1: Using a QPU to increment a number in RAM

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;

operation IncrementInteger (a : Int, nBits : Int) : Int {
    using (register = Qubit[nBits]) {
        let registerLE = LittleEndian(register);
        // Write the integer into the quantum register (using little endian)
        ApplyPauliFromBitString(PauliX, true, IntAsBoolArray(a, nBits), register);
        
        // Perform the increment using library operation
        IncrementByInteger(1, registerLE);
        
        // Read the result from the register and return it
        return MeasureInteger(registerLE);
    }
}

operation IncrementIntegerDemo () : Unit {
    let nBits = 3;
    mutable a = [4, 3, 5, 1];
    Message($"Initial array: {a}");
    
    let indexUpdate = 2;
    set a w/= indexUpdate <- IncrementInteger(a[indexUpdate], nBits);
    Message($"Array after incrementing element {indexUpdate}: {a}");
}

IncrementInteger, IncrementIntegerDemo

In [2]:
%simulate IncrementIntegerDemo

Initial array: [4,3,5,1]
Array after incrementing element 2: [4,3,6,1]


()

### Example 9-2: Using a QPU to increment a number in QRAM

In [3]:
// Example 9-2: Using a QPU to increment a number in QRAM

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;

operation LoadFromQRAM (data : Int[], indexRegister : Qubit[], dataRegister : Qubit[]) : Unit is Adj+Ctl {
    // Mock QRAM loading by using controlled-not gates, with indexRegister as controls
    for (i in 0 .. Length(data) - 1) {
        // Convert the data into a transformation we'll apply on dataRegister to encode it
        let encodeAi = ApplyPauliFromBitString(PauliX, true, IntAsBoolArray(data[i], Length(dataRegister)), _);
        (ControlledOnInt(i, encodeAi))(indexRegister, dataRegister);
    }
}

operation LoadAndIncrementDemo () : Unit {
    let data = [4, 3, 5, 1];
    using ((indexRegister, dataRegister) = (Qubit[2], Qubit[3])) {
        // Prepare index register in superposition
        H(indexRegister[0]);
        X(indexRegister[1]);
        Message("Preparing index register in superposition...");
        DumpMachine();

        // Load the data from QRAM using the index register in superposition as indices
        Message("Loading the data from QRAM...");
        LoadFromQRAM(data, indexRegister, dataRegister);
        DumpMachine();
        
        // Perform the increment using library operation
        Message("Incrementing the data...");
        IncrementByInteger(1, LittleEndian(dataRegister));
        DumpMachine();
        
        ResetAll(indexRegister + dataRegister);
    }
}

LoadAndIncrementDemo, LoadFromQRAM

In [4]:
%simulate LoadAndIncrementDemo

Preparing index register in superposition...


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + 0𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖
|16⟩	0 + 0𝑖
|17⟩	0 + 0𝑖
|18⟩	0 + 0𝑖
|19⟩	0 + 0𝑖
|20⟩	0 + 0𝑖
|21⟩	0 + 0𝑖
|22⟩	0 + 0𝑖
|23⟩	0 + 0𝑖
|24⟩	0 + 0𝑖
|25⟩	0 + 0𝑖
|26⟩	0 + 0𝑖
|27⟩	0 + 0𝑖
|28⟩	0 + 0𝑖
|29⟩	0 + 0𝑖
|30⟩	0 + 0𝑖
|31⟩	0 + 0𝑖

Loading the data from QRAM...


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0.7071067811865476 + 0𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖
|16⟩	0 + 0𝑖
|17⟩	0 + 0𝑖
|18⟩	0 + 0𝑖
|19⟩	0 + 0𝑖
|20⟩	0 + 0𝑖
|21⟩	0 + 0𝑖
|22⟩	0.7071067811865476 + 0𝑖
|23⟩	0 + 0𝑖
|24⟩	0 + 0𝑖
|25⟩	0 + 0𝑖
|26⟩	0 + 0𝑖
|27⟩	0 + 0𝑖
|28⟩	0 + 0𝑖
|29⟩	0 + 0𝑖
|30⟩	0 + 0𝑖
|31⟩	0 + 0𝑖

Incrementing the data...


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	-8.80170503060352E-18 + 5.748367925732736E-18𝑖
|3⟩	-2.6950308208619506E-18 + -5.748367925732736E-18𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	1.3877787807814457E-17 + 4.163336342344338E-17𝑖
|7⟩	-5.551115123125783E-17 + 1.3877787807814448E-17𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	-5.748367925732742E-18 + 6.381970425165232E-34𝑖
|11⟩	0.7071067811865479 + -3.925231146709439E-17𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	-4.163336342344338E-17 + 1.3877787807814454E-17𝑖
|15⟩	5.551115123125784E-17 + 1.0408340855860843E-16𝑖
|16⟩	0 + 0𝑖
|17⟩	0 + 0𝑖
|18⟩	-2.6950308208619506E-18 + -5.748367925732736E-18𝑖
|19⟩	-8.80170503060352E-18 + 5.748367925732736E-18𝑖
|20⟩	0 + 0𝑖
|21⟩	0 + 0𝑖
|22⟩	-5.551115123125783E-17 + 1.3877787807814448E-17𝑖
|23⟩	1.3877787807814457E-17 + 4.163336342344338E-17𝑖
|24⟩	0 + 0𝑖
|25⟩	0 + 0𝑖
|26⟩	0.7071067811865479 + -3.925231146709439E-17𝑖
|27⟩	-5.748367925732742E-18 + 6.381970425165232E-34𝑖
|28⟩	0 + 0𝑖
|29⟩	0 + 0𝑖
|30⟩	5.551115123125784E-17 + 1.0408340855860843E-16𝑖
|31⟩	-4.163336342344338E-17 + 1.3877787807814454E-17𝑖

()

### Example 9-3: Preparing amplitude-encoded vectors

In [5]:
// Example 9-3: Preparing amplitude-encoded vectors

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Preparation;

operation AmplitudeEncodingDemo () : Unit {
    let data = [-1.0, 1.0, 1.0, 5.0, 5.0, 6.0, 6.0, 6.0];
    let N = Length(data);
    
    // Convert array of real data points to an array of ComplexPolar data type
    mutable dataComplexPolar = new ComplexPolar[N];
    for (i in 0 .. N - 1) {
        set dataComplexPolar w/= i <- ComplexAsComplexPolar(Complex(data[i], 0.0));
    }
    
    // Allocate register of right size to amplitude encode vector
    let nQubits = Round(Lg(IntAsDouble(N)));
    using (register = Qubit[nQubits]) {
        let phase = PrepareArbitraryState(dataComplexPolar, LittleEndian(register));
        DumpMachine();
        ResetAll(register);
    }
}

AmplitudeEncodingDemo

In [6]:
%simulate AmplitudeEncodingDemo

|0⟩	-0.07881104062391014 + -4.34442527090695E-17𝑖
|1⟩	0.07881104062391013 + 8.749783189533356E-18𝑖
|2⟩	0.07881104062391 + 3.2374102941011774E-17𝑖
|3⟩	0.39405520311955056 + -1.4678941006134315E-16𝑖
|4⟩	0.39405520311955056 + -2.1874457973833392E-17𝑖
|5⟩	0.47286624374346076 + 2.6249349568600075E-17𝑖
|6⟩	0.4728662437434605 + 0𝑖
|7⟩	0.4728662437434607 + 0𝑖

()